# CP213: Model Solutions for Tutorial Notebook 9


## Question 1

A spherical projectile of diameter $D=42.67\,{\rm mm}$ and mass
$45.9\,{\rm g}$ initially located at the origin is struck so it has an
initial speed $v_0=70\,{\rm m\,s}^{-1}$ directed at an angle of
$\theta=45^\circ$ from the horizontal (aligned with the
$x$-direction).  It travels a gravitational field $g=9.81\,{\rm
m\,s}^{-2}$ which points in the negative $y$-direction.

The drag force $F_d$ on the sphere is given by
\begin{align*}
F_d &= C_d A \frac{1}{2}\rho v^2
\end{align*}
where $C_d=0.44$ is the drag coefficient for the sphere, $A=\pi D^2/4$
is the cross-sectional area of the projectile, $\rho=1.2\,{\rm
kg\,m^{-3}}$ is the density of air, and $v$ is the speed of the
sphere.

1. Perform a $x$-momentum and $y$-momentum balance to develop a pair
   of differential equations that govern how the $x$- and $y$-velocity of the
   projectile vary with time.

2. Use the Euler method to solve the differential equations derived
   above.  Benchmark your code against the case where there is no drag
   and the equations can be solved analytically.  In addition, you can
   also compare your solution with that from
   [scipy.integrate.solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).

3. Plot the trajectory (i.e. $y(t)$ vs $x(t)$) of the projectile with
   time, from $t=0$ until it hits the ground (located at $y=0$).  You
   will need to include the differential equations that relate the
   $x$- and $y$-coordinates of the projectile to its velocity.

4. What is the initial angle $\theta$ where the projectile will travel
   the furthest in the $x$-direction?
   
5. If the drag coefficient $C_d$ varies with the speed of the
   projectile as [Goossens 2019](https://dx.doi.org/10.1016/j.powtec.2019.04.075)
\begin{align*}
C_d &\approx \frac{24}{\rm Re} + 0.44
\end{align*}
where the Reynolds number Re is defined as
\begin{align*}
{\rm Re}
&= \frac{D \rho v}{\mu}
\end{align*}
with $\mu=1.8\times10^{-5}\,{\rm Pa\,s}$, redetermine the trajectory
of the projectile.


Below is some sample code that solves the [Lotka-Volterra equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations)
using the `solve_ivp` function from `scipy`
\begin{align*}
\frac{dy_0}{dt} &= ay_0 - by_0y_1
\\
\frac{dy_1}{dt} &= -cy_1 + dy_0y_1
\end{align*}
where $a=1.5$, $b=1$, $c=3$, and $d=1$.  The initial conditions are
$y_0(0)=10$, and $y_1(0)=5$.


In [ ]:
import numpy as np
import pylab as plt
from scipy.integrate import solve_ivp


def dydt(t, y):
 
    a=1.5
    b=1
    c=3
    d=1

    f = []
    f.append( a*y[0] - b*y[0]*y[1])
    f.append(-c*y[1] + d*y[0]*y[1])
    
    return f

t_init = 0.0
t_final = 15.0
t_dat = np.linspace(t_init, t_final, 100)
y_init = [10.0, 5.0]
solution = solve_ivp(dydt, (t_init, t_final), y_init, t_eval=t_dat)

#print(solution)
plt.plot(solution.t, solution.y[0], label=r'$y_0(t)$')
plt.plot(solution.t, solution.y[1], label=r'$y_1(t)$') 

plt.legend()
plt.xlabel(r'$t$')
plt.show()


## Model solutions for Q1

### Part 1

This problem is closely related to the second question in tutorial 1,
where we examined the trajectory in the absence of any drag force.
The main complication in this problem is that the drag force links the
$x$-component of the projectile's velocity to the $y$-component of its
velocity; consequently, these differential equations cannot be solved
independently of each other.

The drag force $F_d$ always acts in a direction exactly opposite of
the velocity vector of the projectile.  That is, the drag force has a
magnitude $F_d$ and points in the direction of ${\bf v}$.  The unit
vector $\hat{\bf v}$ that points in the direction of the velocity is
defined as $\hat{\bf v}={\bf v}/v$, where $v$ is the magnitude of the
velocity vector (i.e. $v=\sqrt{{\bf v}\cdot{\bf v}}$).  With this, we
can then write:
\begin{align*}
{\bf F}_d &= F_d \, \hat{\bf v}
,
\end{align*}
which implies that $x$- and $y$-components of the drag force vector are
\begin{align*}
F_{d,x} &= -F_d \frac{v_x}{v}
\\
F_{d,y} &= -F_d \frac{v_y}{v}
\end{align*}

We consider a time interval from $t$ to $t+\Delta t$.  The
$x$-momentum balance is:
\begin{align*}
m v_x(t+\Delta t) - m v_x(t)
&= - F_d \frac{v_x(t)}{v(t)}  \Delta t
\\
\frac{m v_x(t+\Delta t) - m v_x(t)}{\Delta t}
&= - F_d \frac{v_x(t)}{v(t)}  
\\
\frac{d}{d t}[m v_x(t)]
&= - F_d \frac{v_x(t)}{v(t)}  
\\
\frac{d v_x(t)}{d t}
&= - \frac{F_d}{m} \frac{v_x(t)}{v(t)}  
\end{align*}

The $y$-momentum balance is:
\begin{align*}
m v_y(t+\Delta t) - m v_y(t)
&= - F_d \frac{v_y(t)}{v(t)}  \Delta t
- m g \Delta t
\\
\frac{m v_y(t+\Delta t) - m v_y(t)}{\Delta t}
&= - F_d \frac{v_y(t)}{v(t)}  
- m g 
\\
\frac{d}{dt}[m v_y(t)]
&= - F_d \frac{v_y(t)}{v(t)}  
- m g 
\\
\frac{d v_y(t)}{dt}
&= - \frac{F_d}{m} \frac{v_y(t)}{v(t)} - g 
\end{align*}



### Part 2

We write a simple Python code to implement the Euler method to solve
the set of first order differential equations, which can be written in
the form
\begin{align*}
\frac{dy_0(t)}{dt}
&= f_0(t, y_0(t), y_1(t))
\\
\frac{dy_1(t)}{dt}
&= f_1(t, y_0(t), y_1(t))
\end{align*}
The Euler method then used the following approximate stepping
procedure to solve the equations:
\begin{align*}
y_0(t+\Delta t)
&\approx y_0(t) + f_0(t, y_0(t), y_1(t)) \Delta t
\\
y_1(t+\Delta t)
&\approx y_1(t) + f_1(t, y_0(t), y_1(t))  \Delta t
,
\end{align*}
where $\Delta t$ is the time step.  Note that we need to ensure that
the time step is sufficiently small that we can obtain an accurate
solution.

The code is given below.  Note that the function `dydt` returns the
derivatives of the velocity components in a list:


In [ ]:
import numpy as np
import pylab as plt

def dydt(t, y):
    vx, vy  = y    # unpack the list y to vx and vy
    v2 = vx**2 + vy**2
    v = np.sqrt(v2)
    Fd = Cd * A * 0.5*rho*v**2  # magnitude of the drag force
    dvx = ( - Fd*vx/v       ) / m
    dvy = ( - Fd*vy/v - m*g ) / m
    return [dvx, dvy]    # return the derivatives of the velocity with time
    

Cd = 0.44       # drag coefficient
rho = 1.2       # density of air / kg m^{-3}
D = 42.67e-3    # diameter of projectile / m
m = 45.9e-3     # mass of projectile / kg
theta_degrees = 45.0
theta = theta_degrees * np.pi/180.0
v0 = 70.0       # initial speed / m s^{-1}
g = 9.81        # acceleration due to gravity / m s^{-2}

A = np.pi*D**2/4.0

dt = 0.01       # time step / s
t_init = 0.0    # initial time / s
t_final = 15.0  # final time / s
t_dat = np.arange(t_init, t_final, dt)
vx_dat = []
vy_dat = []
t = 0.0
vx = v0*np.cos(theta)  # initial x-velocity / m s^{-1}
vy = v0*np.sin(theta)  # initial y-velocity / m s^{-1}
for t in t_dat:
    dvx, dvy = dydt(t, [vx, vy])    

    vx_new = vx + dvx * dt
    vy_new = vy + dvy * dt
    
    vx = vx_new
    vy = vy_new
    
    vx_dat.append(vx)
    vy_dat.append(vy)

plt.plot(t_dat, vx_dat, label=r'v_x(t)')
plt.plot(t_dat, vy_dat, label=r'v_y(t)')
plt.legend()
plt.xlabel(r'time / s')
plt.ylabel(r'speed / m s$^{-1}$')

plt.show()


Let's check that this code works properly by making sure that it reproduces the known analytical solution when there is no drag on the projectile.  In this case, we know that $v_x(t)=v_0\sin\theta$ is constant with time, and $v_y(t)=v_0\cos\theta - g t$.  

In [ ]:
import numpy as np
import pylab as plt

def dydt(t, y):
    vx, vy  = y
    v2 = vx**2 + vy**2
    v = np.sqrt(v2)
    Fd = Cd * A * 0.5*rho*v**2
    dvx = ( - Fd*vx/v       ) / m
    dvy = ( - Fd*vy/v - m*g ) / m
    return [dvx, dvy]
    

Cd = 0.0    
rho = 1.2
D = 42.67e-3
m = 45.9e-3
theta_degrees = 45.0
theta = theta_degrees * np.pi/180.0
v0 = 70.0
g = 9.81

A = np.pi*D**2/4.0

dt = 0.01
t_init = 0.0
t_final = 15.0
t_dat = np.arange(t_init, t_final, dt)
vx_dat = []
vy_dat = []
t = 0.0
vx = v0*np.cos(theta)
vy = v0*np.sin(theta)
for t in t_dat:
    dvx, dvy = dydt(t, [vx, vy])    

    vx_new = vx + dvx * dt
    vy_new = vy + dvy * dt
    
    vx = vx_new
    vy = vy_new
    
    vx_dat.append(vx)
    vy_dat.append(vy)

    
    
plt.plot(t_dat, vx_dat, marker='x', color='black', ls='None', label=r'$v_x(t)$')
plt.plot(t_dat, vy_dat, marker='x', color='red', ls='None', label=r'$v_y(t)$')


vx_exact = [v0*np.cos(theta) for t in t_dat]
vy_exact = [v0*np.sin(theta)- g*t for t in t_dat]
plt.plot(t_dat, vx_exact, color='black')
plt.plot(t_dat, vy_exact, color='red')

plt.legend()
plt.xlabel(r'time / s')
plt.ylabel(r'speed / m s$^{-1}$')

plt.show()


The symbols are the numerical solution obtained from the Euler method, and the lines are the results of the analytical solution.  We see that the numerical solution reproduces the analytical solution in this case, so this gives us some confidence in our Euler method code.

Now let's compare our code with the solution obtained from `solve_ivp` from the library `scipy.integrate`.

In [ ]:
import numpy as np
import pylab as plt
from scipy.integrate import solve_ivp

def dydt(t, y):
    vx, vy  = y
    v2 = vx**2 + vy**2
    v = np.sqrt(v2)
    Fd = Cd * A * 0.5*rho*v**2
    dvx = ( - Fd*vx/v       ) / m
    dvy = ( - Fd*vy/v - m*g ) / m
    return [dvx, dvy]
    

Cd = 0.44
rho = 1.2
D = 42.67e-3
m = 45.9e-3
theta_degrees = 45.0
theta = theta_degrees * np.pi/180.0
v0 = 70.0
g = 9.81

A = np.pi*D**2/4.0

dt = 0.01
t_init = 0.0
t_final = 15.0
t_dat = np.arange(t_init, t_final, dt)
vx_dat = []
vy_dat = []
t = 0.0
vx = v0*np.cos(theta)
vy = v0*np.sin(theta)
for t in t_dat:
    dvx, dvy = dydt(t, [vx, vy])    

    vx_new = vx + dvx * dt
    vy_new = vy + dvy * dt
    
    vx = vx_new
    vy = vy_new
    
    vx_dat.append(vx)
    vy_dat.append(vy)

    
    
plt.plot(t_dat, vx_dat, marker='x', color='black', ls='None', label=r'$v_x(t)$')
plt.plot(t_dat, vy_dat, marker='x', color='red', ls='None', label=r'$v_y(t)$')


t_init = 0.0
t_final = 15.0
t_dat = np.linspace(t_init, t_final, 100)
y_init = [v0*np.cos(theta), v0*np.sin(theta)]
solution = solve_ivp(dydt, (t_init, t_final), y_init, t_eval=t_dat)

#print(solution)
plt.plot(solution.t, solution.y[0], color='black')
plt.plot(solution.t, solution.y[1], color='red')




plt.legend()
plt.xlabel(r'time / s')
plt.ylabel(r'speed / m s$^{-1}$')

plt.show()    


In this case, the solid lines are from `solve_ivp`, and we see that there is good agreement between the two codes.

### Part 3

To determine the trajectory of the projectile, we simple need to add
two additional differential equations that relate the rate of change
of the position of the projectile with its velocity:
\begin{align*}
\frac{dx}{dt} &= v_x
\\
\frac{dy}{dt} &= v_y
\end{align*}
We now have four first order differential equations to solve:
\begin{align*}
\frac{dx}{dt} &= v_x
\\
\frac{dy}{dt} &= v_y
\\
\frac{d v_x(t)}{d t}
&= - \frac{F_d}{m} \frac{v_x(t)}{v(t)}  
\\
\frac{d v_y(t)}{dt}
&= - \frac{F_d}{m} \frac{v_y(t)}{v(t)} - g 
.
\end{align*}
We will use `solve_ivp` to solve these equations below.



In [ ]:
import numpy as np
import pylab as plt
from scipy.integrate import solve_ivp


def dydt(t, y):
    x, y, vx, vy  = y
    v2 = vx**2 + vy**2
    v = np.sqrt(v2)
    Fd = Cd * A * 0.5*rho*v**2
    dx = vx
    dy = vy
    dvx = (- Fd*vx/v       ) / m
    dvy = (- Fd*vy/v - m*g ) / m
    return [dx, dy, dvx, dvy]


Cd = 0.44
rho = 1.2    # density of air
D = 42.67e-3
m = 45.9e-3
theta_degrees = 45.0
theta = theta_degrees * np.pi/180.0
v0 = 70.0
g = 9.81

A = np.pi*D**2/4.0


t_init = 0.0
t_final = 15.0
t_data = np.linspace(t_init, t_final, 100)
y_init = [0.0, 0.0, v0*np.cos(theta), v0*np.sin(theta)]
solution = solve_ivp(dydt, (t_init, t_final), y_init, t_eval=t_data)


plt.plot(solution.y[0], solution.y[1], label=r'$y_1(t)$') 
plt.ylim([0.0, 100])
plt.xlabel(r'distance / m')
plt.ylabel(r'height / m')

plt.show()


We see that the trajectory of the projectile is somewhat lopsided, in comparison to the case where this is no drag due to air where the trajectory is a symmetric parabola.

### Part 4

In this problem, we will simply adjust the angle to determine the value which gives use the longest distance.  Because we will need to repeat the calculations for different values of $\theta$, it is convenient to define a function takes $\theta$ as an argument and returns the trajectory.

In [ ]:
import numpy as np
import pylab as plt
from scipy.integrate import solve_ivp


def dydt(t, y):
    x, y, vx, vy  = y
    v2 = vx**2 + vy**2
    v = np.sqrt(v2)
    Fd = Cd * A * 0.5*rho*v**2
    dx = vx
    dy = vy
    dvx = (- Fd*vx/v       ) / m
    dvy = (- Fd*vy/v - m*g ) / m
    return [dx, dy, dvx, dvy]

def get_trajectory(theta_degrees):

    theta = theta_degrees * np.pi/180.0

    t_init = 0.0
    t_final = 10.0
    t_data = np.linspace(t_init, t_final, 100)
    y_init = [0.0, 0.0, v0*np.cos(theta), v0*np.sin(theta)]
    solution = solve_ivp(dydt, (t_init, t_final), y_init, t_eval=t_data)

    return solution.y[0], solution.y[1]


Cd = 0.44
rho = 1.2        # density of air / kg m^{-3}
mu = 1.8e-5      # viscosity of air / Pa s
D = 42.67e-3
A = np.pi*D**2/4.0
m = 45.9e-3
v0 = 70.0
g = 9.81
    
for theta_degrees in [20, 30, 32, 45, 50]:
    x_dat, y_dat = get_trajectory(theta_degrees)
    plt.plot(x_dat, y_dat, label=r'$\theta=$'+f'{theta_degrees}'+r'$^\circ$') 


#plt.ylim([0.0, 100])
plt.legend()
plt.xlabel(r'distance / m')
plt.ylabel(r'height / m')
plt.ylim([0.0, 100])

plt.show()


### Part 5

In this problem, we just need to include a more complicated expression for the magnitude of the drag force.  As you can see, this does not require too many changes to the code.

In [ ]:
import numpy as np
import pylab as plt
from scipy.integrate import solve_ivp


def dydt(t, y):
    x, y, vx, vy  = y
    v2 = vx**2 + vy**2
    v = np.sqrt(v2)
    Re = D*rho*v/mu
    Cd = 24/Re + 0.44
    Fd = Cd * A * 0.5*rho*v**2
    dx = vx
    dy = vy
    dvx = (- Fd*vx/v       ) / m
    dvy = (- Fd*vy/v - m*g ) / m
    return [dx, dy, dvx, dvy]

def get_trajectory(theta_degrees):

    theta = theta_degrees * np.pi/180.0

    t_init = 0.0
    t_final = 10.0
    t_data = np.linspace(t_init, t_final, 100)
    y_init = [0.0, 0.0, v0*np.cos(theta), v0*np.sin(theta)]
    solution = solve_ivp(dydt, (t_init, t_final), y_init, t_eval=t_data)

    return solution.y[0], solution.y[1]


rho = 1.2    # density of air
mu = 1.8e-5
D = 42.67e-3
A = np.pi*D**2/4.0
m = 45.9e-3
v0 = 70.0
g = 9.81

for theta_degrees in [20, 30, 32, 45, 50]:
    x_dat, y_dat = get_trajectory(theta_degrees)
    plt.plot(x_dat, y_dat, label=r'$\theta=$'+f'{theta_degrees}'+r'$^\circ$') 


#plt.ylim([0.0, 100])
plt.legend()
plt.xlabel(r'distance / m')
plt.ylabel(r'height / m')
plt.ylim([0.0, 100])

plt.show()
